In [ ]:
# Load libraries 
import pandas as pd
import matplotlib.pyplot as plt
import folium
import folium.plugins
import branca
import branca.colormap as cm
#import shapefile
from shapely.geometry import shape, Point

In [ ]:
# Read wealth index file 
dhs_wealth_index = pd.read_csv("s3://w210-poverty-mapper/dhs_data/transformed_data/dhs_wealth_index.csv")

In [ ]:
# Check number of rows and cols
dhs_wealth_index.shape

In [ ]:
dhs_wealth_index.describe()

In [ ]:
# Count clusters by country
dhs_wealth_index.groupby("country")["cluster"].nunique()

In [ ]:
# Count clusters with lat = 0 & lon = 0
dhs_wealth_index[(dhs_wealth_index["lat"] == 0) & (dhs_wealth_index["lon"] == 0) ].shape[0]

In [ ]:
# Plot histograms of wealth index by country
fig = plt.figure(figsize=(12,6))

country_list = [c for c in dhs_wealth_index["country"].unique()]
#country_list

for country, num in zip(country_list, range(1,7)):
    ax = fig.add_subplot(2,3,num)
    ax.hist(dhs_wealth_index["wealth_index"][dhs_wealth_index["country"] == country])
    ax.set_title(country)
    ax.set_ylim([0, 400])
    ax.set_xlim([0, 100])

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
# Load density data
density = pd.read_csv("s3://w210-poverty-mapper/modeling/metadata/source_data/meta_data_full_updated_density_new_full_value_LZ.csv")

In [ ]:
density.head()

In [ ]:
# Plot histograms of wealth index by country
fig = plt.figure(figsize=(12,6))

country_list = [c for c in density["countries"].unique()]
#country_list

for country, num in zip(country_list, range(1,7)):
    ax = fig.add_subplot(2,3,num)
    ax.hist(density["Density"][density["countries"] == country])
    ax.set_title(country)
    ax.set_ylim([0, 1000000])
    ax.set_xlim([0, 1000])

plt.tight_layout()
plt.show()

In [ ]:
# Create base map
colormap = cm.LinearColormap(colors=["red", "orange", "yellow", "lightgreen", "green"], index=[0, 25, 50, 75, 100],vmin=0,vmax=100)

map = folium.Map(location=[-6, 96], 
                 zoom_start=2.5, 
                 control_scale=True)

folium.TileLayer("OpenStreetMap").add_to(map)
folium.TileLayer("Stamen Terrain").add_to(map)
folium.TileLayer("CartoDB positron").add_to(map)
folium.LayerControl(position="topleft").add_to(map)

In [ ]:
# Add cluster GPS coordinates with pop-up
for index, location_info in dhs_wealth_index.iterrows():
    folium.Circle([location_info["lat"],
                   location_info["lon"]], 
                  radius=2,
                  color=colormap(location_info["wealth_index"]),
                  popup=location_info[["country", "cluster", "wealth_index"]]).add_to(map)

map.add_child(colormap)